# Getting Started with SQL 

#### TAMÁS BUDAVÁRI – AUGUST 2016 
 
Databases are easy: they consist of tables, which in turn have columns – just like the files with which you 
are probably used to working. The difference is that you don’t have to think about how to read and write 
the files but can focus on the questions you try to answer. The following code snippets and exercises will 
teach you the basics of expressing your questions in the Structure Query Language, or SQL for short. SQL 
is a standard language and most of these commands will work on any relational database but there are 
minor differences in dialects. 
 
The database you will be querying stores a collection of measurements of some (X,Y) quantities. There is 
a relation between them but the Y measurements are noisy. Our database is synthetic but carries several 
aspects of real measurements: separate instruments, multiple students and runs of measurements in 
different observational domains. The relevant tables are Data, Runs, Instruments and Users. The names 
should be suggestive of their contents.

You have tried out some SQL queries using the online <a href="https://skyserver.sdss.org/CasJobs/">CasJobs web application</a>. Here we use many of these same queries as examples for accessing a set of relational databases from within Jupyter notebooks running python inside a 
<a href="https://apps.sciserver.org/compute/">SciServer/Compute</a> container.


## SciServer.CasJobs
From  a python notebook in SciServer/Compute you can submit SQL queries to the same service that runs queries from the CasJobs webpage. The advantage is that you can directly access the result and visualize it or analyze it in any way you want.

To do so you must import a library that is available on all the SciServer Compute images.

In [1]:
import SciServer.CasJobs as cj

## The First Queries 
To get all (X,Y) values from the table Data, you could use the following SQL:

``` sql
-- never run queries like this 
select X, Y 
from Data 
```

But don’t do it! First you should always think about what will happen. 

The table might have hundreds of millions of rows! Do you really want all that data dumped on you? Try 
the next set of commands to see their effects and understand how they work. Consider them as 
illustrations accompanying the lecture and ask questions if something is not clear! 

In [2]:
sql="""
-- have a quick peek  
select top 5 X, Y 
from Data 
"""

To execute this query you will use the SciServer.CasJobs library, which we aliased to 'cj' in the import statement.
Its 'executeQuery' function submits the specified sql to the specified context<br/>
As written here this query will be executed "synchronously" and return the result as a <a href="https://pandas.pydata.org/">pandas</a> <a href="http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe">DataFrame</a> that will be printed at the end of the cell.

In [3]:
df=cj.executeQuery(sql=sql,context="IntroSQL")
df

,X,Y
0,0.145097,-1.131867
1,0.200108,-0.337185
2,0.405262,-0.246609
3,0.338955,-1.156640
4,0.089844,-1.062386


If you only want to inspect the result you do not need to store it in a variable. And note that you do not need to be explicit about the variables. See <a href="http://www.sciserver.org/docs/sciscript-python/">docs</a> for documentation on all the SciServer python modules. In particular <a href="http://www.sciserver.org/docs/sciscript-python/SciServer.html#module-SciServer.CasJobs">SciServer.CasJobs</a> for the CasJobs module.

In [7]:
# sorting to peek at the extremes 
sql="""
select top 3 X, Y 
from Data 
order by X 
"""
cj.executeQuery(sql,"IntroSQL")

,X,Y
0,0.001319,-1.121337
1,0.001944,-0.888723
2,0.002475,-0.770705


In [9]:
sql="""
-- filtering with formulas and functions 
select top 5 x 
from Data 
where 2*y between -sin(x) and x 
order by RunID desc, y desc 
"""
cj.executeQuery(sql,"IntroSQL")

,x
0,0.546278
1,0.434254
2,0.438969
3,0.501610
4,0.430480


The last query showed you that you can comment your SQL code by using the following formats 
 <pre>
-- single-line comments go after -- 
 
/* multi-line comments are  
   like this one 
*/ 
</pre> 
 

# NOT TRUE:
### SAVE YOUR WORK! 
The website you are using will not save your queries. If you would 
like to keep them for future reference, open a text editor to  cut & 
paste the relevant lines into a file that you can regularly save. 
 
IF you save your notebook in a volume in your private /Storage/&lt;username&gt; area, for example /persistent, your notebook will be backed up and available in other containers as well.


## Aggregation
Consider getting only the relevant information from the database and not everything. Run the following 
commands and see what the different constructs achieve: 

In [13]:
sql="""
-- counting 
select COUNT(ID) as N, COUNT(RunID), COUNT(distinct RunID) 
from Data 
"""
cj.executeQuery(sql,"IntroSQL")

,N,Column1,Column2
0,546,546,10


In [14]:
sql="""
-- aggregates in general 
select COUNT(id), SUM(x), AVG(y), STDEV(x-y) 
from Data 
where Y>0 
"""
cj.executeQuery(sql,"IntroSQL")

,Column1,Column2,Column3,Column4
0,230,127.140058,1.509137,1.403175


In [15]:
sql="""
-- grouping data 
select RunID, MIN(x), MAX(x), MIN(y), MAX(Y) 
from Data 
group by RunID 
order by AVG(X) desc 
"""
cj.executeQuery(sql,"IntroSQL")

,RunID,Column1,Column2,Column3,Column4
0,101,0.505333,0.994198,-0.086970,2.975272
1,109,0.356064,0.879783,-1.207412,1.644311
2,104,0.255726,0.789319,-2.374107,1.774286
3,102,0.242926,0.773327,-0.935795,1.074530
4,105,0.202550,0.797702,-8.014773,6.073653
5,108,0.302871,0.599753,-0.700361,0.266618
6,106,0.202057,0.799781,-5.970850,4.230965
7,103,0.004561,0.881169,-7.480040,5.935146
8,100,0.017204,0.492729,-1.619121,0.487096
9,107,0.001319,0.097952,-1.163017,-0.693459


In [16]:
sql="""
-- having: contraints on aggregates 
select RunID, MIN(x), MAX(x), MIN(y), MAX(Y) 
from Data 
where X>0.2         -- filtering on the input 
group by RunID 
having MAX(Y) < 0   -- filtering on aggregate 
order by AVG(X) desc 
"""
cj.executeQuery(sql,"AstroinformIntroSQLatics2018")

,RunID,Column1,Column2,Column3,Column4


In [18]:
sql="""
-- or  
select RunID, MIN(x), MAX(x), MIN(y), MAX(Y) 
from Data 
where X>0.2 
group by RunID 
having COUNT(*) > 30 
order by AVG(X) desc 
"""
cj.executeQuery(sql,"IntroSQL")

,RunID,Column1,Column2,Column3,Column4
0,103,0.207009,0.881169,-5.723222,5.935146
1,105,0.202550,0.797702,-8.014773,6.073653
2,108,0.302871,0.599753,-0.700361,0.266618
3,106,0.202057,0.799781,-5.970850,4.230965


In [19]:
sql="""
-- number of measurements in each run 
select RunID, COUNT(*) 
from Data 
group by RunID 
order by 2 desc 
"""
cj.executeQuery(sql,"IntroSQL")

,RunID,Column1
0,103,99
1,105,99
2,106,99
3,108,73
4,107,49
5,102,30
6,109,27
7,100,25
8,101,25
9,104,20


In [20]:
sql="""
-- rounding is easy 
select top 100 X, ROUND(X*X,2) from Data 
"""
cj.executeQuery(sql,"IntroSQL")

,X,Column1
0,0.145097,0.02
1,0.200108,0.04
2,0.405262,0.16
3,0.338955,0.11
4,0.089844,0.01
5,0.131127,0.02
6,0.465117,0.22
7,0.049429,0.00
8,0.391001,0.15
9,0.357276,0.13


In [21]:
sql="""
-- building a histogram 
select ROUND(x,2) as X, COUNT(*) as N 
from Data 
group by ROUND(x,2) 
order by 1 
"""
cj.executeQuery(sql,"IntroSQL")

,X,N
0,0.00,4
1,0.01,8
2,0.02,6
3,0.03,6
4,0.04,6
5,0.05,8
6,0.06,8
7,0.07,5
8,0.08,5
9,0.09,11


You can use the same <pre> SELECT ... INTO ...</pre> pattern to save results in your MyDB. Make sure though that a table with this name does not already exist!

In [34]:
sql="""
-- custom bin size using a variable and save the results 
declare @bin float = 0.016 
select ROUND(x/@bin,0)*@bin as X, COUNT(*) as Cts 
into AHistogram  -- name of new table 
from Data 
group by ROUND(x/@bin,0)*@bin 
order by ROUND(x/@bin,0)*@bin 
"""
cj.executeQuery(sql,"IntroSQL")

,Rows Affected
0,61


Check that the table was indeed created in your MyDB in the CasJobs UI at https://skyserver.sdss.org/CasJobs/MyDB.aspx

## Asking Questions using SQL 

You’ll see that every question you have about the data will nicely translate to commands. For example, 
let’s consider the following question: Who ran the first measurement? The following 3 select statements 
will get you an answer. Note, you can run all three queries using a single execution. When more than 1 result is returned from the database, the result is a list of data frames: 

In [26]:
sql="""
select top 1 RunID from Data order by ID -- 100 
select UserID from Runs where RunID=100 -- 12 
select * from Users where UserID=12 
"""
dfs=cj.executeQuery(sql,"IntroSQL")

Loop over the elements in the result, each of which is a DataFrame:

In [33]:
for idx, df in enumerate(dfs):
    print(idx,":")
    print(df)

0 :
   RunID
0    100
1 :
   UserID
0      12
2 :
   UserID        Name  AdvisorID
0      12  Hugo First         10


What did we really mean by “first” measurement? Any other definition to use? 
 
Nested queries can combine multiple searches into one request. Run and analyze the following queries 
and their results: 

In [35]:
sql="""
-- nested queries 
select UserID 
from Runs 
where RunID = (select top 1 RunID from Data order by ID) 
 
-- doubly so 
select * from Users 
where UserID = ( 
  select UserID from Runs 
  where RunID = (select top 1 RunID from Data order by ID) 
) 
 
-- whole set of runs using the 'in' keyword 
select UserID 
from Runs 
where RunID in (select top 1 RunID  
                from Data order by ID) 
"""
dfs=cj.executeQuery(sql,"IntroSQL")

In [36]:
for idx, df in enumerate(dfs):
    print(idx,":")
    print(df)

0 :
   UserID
0      12
1 :
   UserID        Name  AdvisorID
0      12  Hugo First         10
2 :
   UserID
0      12


Combining tables is where relational database engines really shine. The terminology is “joining tables”. 
Here  are  different  implementations  of  similar  questions.  Make  sure  you  understand  these  queries 
because they will be important: 
 

In [37]:
sql="""
-- inner join (old style)  
select u.Name, r.RunID, r.Xmax - r.Xmin 
from Runs r, Users u -- aliases are convenient 
where r.UserID=u.UserID  
-- risk of forgetting a constraint when many tables 
"""
cj.executeQuery(sql,"IntroSQL")

,Name,RunID,Column1
0,Hugo First,100,0.5
1,Hugo First,101,0.5
2,Hammond Eggs,102,0.6
3,Gil T. Azell,103,0.9
4,Holly Wood,104,0.6
5,Gil T. Azell,105,0.6
6,Gil T. Azell,106,0.6
7,Holly Wood,107,0.1
8,Holly Wood,108,0.3
9,Levy Tate,109,0.6


In [38]:
sql="""
-- inner join (preferred) 
select u.Name, r.RunID, r.Xmax - r.Xmin 
from Runs r 
     join Users u on u.UserID=r.UserID 
"""
cj.executeQuery(sql,"IntroSQL")

,Name,RunID,Column1
0,Hugo First,100,0.5
1,Hugo First,101,0.5
2,Hammond Eggs,102,0.6
3,Gil T. Azell,103,0.9
4,Holly Wood,104,0.6
5,Gil T. Azell,105,0.6
6,Gil T. Azell,106,0.6
7,Holly Wood,107,0.1
8,Holly Wood,108,0.3
9,Levy Tate,109,0.6


In [39]:
sql="""-- or explicitly  
select u.Name, r.RunID, r.Xmax - r.Xmin 
from Runs r 
     inner join Users u on u.UserID=r.UserID 
"""
cj.executeQuery(sql,"IntroSQL")

,Name,RunID,Column1
0,Hugo First,100,0.5
1,Hugo First,101,0.5
2,Hammond Eggs,102,0.6
3,Gil T. Azell,103,0.9
4,Holly Wood,104,0.6
5,Gil T. Azell,105,0.6
6,Gil T. Azell,106,0.6
7,Holly Wood,107,0.1
8,Holly Wood,108,0.3
9,Levy Tate,109,0.6


In [40]:
sql=""" 
-- list of users with measurements 
select distinct u.Name 
from Runs r 
     join Users u on u.UserID=r.UserID      
"""
cj.executeQuery(sql,"IntroSQL")

,Name
0,Gil T. Azell
1,Hammond Eggs
2,Holly Wood
3,Hugo First
4,Levy Tate


A different kind of join is used less frequently to look at all combinations of rows in the specified tables. 
Compare the following queries to the previous ones and run them to see the differences in the results: 
 

In [42]:
sql="""
-- cross join (old style)  
select u.Name, r.RunID, r.Xmax - r.Xmin 
from Runs r, Users u 
-- same as old-style inner join w/o contraint 
"""
cj.executeQuery(sql,"IntroSQL")

,Name,RunID,Column1
0,Andy Structible,100,0.5
1,Andy Structible,101,0.5
2,Andy Structible,102,0.6
3,Andy Structible,103,0.9
4,Andy Structible,104,0.6
5,Andy Structible,105,0.6
6,Andy Structible,106,0.6
7,Andy Structible,107,0.1
8,Andy Structible,108,0.3
9,Andy Structible,109,0.6


In [43]:
sql=""" 
-- cross join: explicitly 
select u.Name, r.RunID, r.Xmax - r.Xmin 
from Runs r cross join Users u 
"""
cj.executeQuery(sql,"IntroSQL")

,Name,RunID,Column1
0,Andy Structible,100,0.5
1,Andy Structible,101,0.5
2,Andy Structible,102,0.6
3,Andy Structible,103,0.9
4,Andy Structible,104,0.6
5,Andy Structible,105,0.6
6,Andy Structible,106,0.6
7,Andy Structible,107,0.1
8,Andy Structible,108,0.3
9,Andy Structible,109,0.6


In [44]:
sql=""" 
-- compare the size of the result set with these 
select COUNT(*) from users 
"""
cj.executeQuery(sql,"IntroSQL")

,Column1
0,7


In [41]:
sql="""
select COUNT(*) from runs 
-- all combinations 
"""
cj.executeQuery(sql,"IntroSQL")

,Column1
0,10
